In [ ]:
# import defaults
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
# options for dealing with websites that populate while a user scrolls
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
driver = webdriver.Chrome() #options=chrome_options

# link to Google News Archive search for "GameStop" from 04JAN2021 - 31DEC2021
driver.get("https://www.google.com/search?q=GameStop&sca_esv=54b6785aa301342b&tbs=cdr:1,cd_min:1/4/2021,cd_max:12/31/2021&tbm=nws&source=lnms&sa=X&ved=2ahUKEwjjn4HW1qaEAxV5FFkFHYNAAZ4Q0pQJegQIBhAG&biw=1600&bih=731&dpr=1")

# Wait for the initial content to load
time.sleep(5)

# essentially a timer, restricts the number of pages the Selenium web crawler can access
counter = 0
# lists to store all document titles and publication dates
all_titles = []
all_dates = []

# Scroll down to load more articles
html = driver.find_element(By.TAG_NAME, 'html')
for _ in range(10):  # Adjust the range depending on how much you need to scroll
    html.send_keys(Keys.END)
    

#Source: https://sqa.stackexchange.com/questions/50747/selenium-python-write-click-next-page-loop
#set counter to 25; Google typically allows only 30 pages of results for this type of query
#counter is not synced with Selenium operations or timer
#Google also occassionally changes the XPATH to the next button
while counter <=25:
    # Scrape the article titles
    # get all the article titles and publication dates
    articles = driver.find_elements(By.XPATH, "//body//a[@class='WlydOe']//div[@class='n0jPhd ynAwRc MBeuO nDgy9d']")
    pub_dates = driver.find_elements(By.XPATH, "//div[@class='SoAPf']/div[@class='OSrXXb rbYSKb LfVVr']/span")
    titles = [article.text for article in articles]
    dates = [date.text for date in pub_dates]
    # append titles and publication dates to lists
    for title in titles:
        all_titles.append(title)
    for date in dates:
        all_dates.append(date)
    #Get the "Next" button using the XPath
    next_button_xpath = "/html/body[@id='gsr']/div[@id='main']/div[@id='cnt']/div[@id='rcnt']/div[@id='center_col']/div[@id='botstuff']/div/div[3]/table[@class='AaVjTc']/tbody/tr/td[@class='d6cvqb BBwThe'][2]/a[@id='pnnext']/span[2]"
    next_button = driver.find_element(By.XPATH, next_button_xpath)
    
    # Click the next button at the bottom of the page
    next_button.click()
    # iterate counter
    counter +=1
    time.sleep(2)  # Add delay between scrolls for the page to load
    

# Close the browser
driver.quit()

In [ ]:
len(all_titles)

In [ ]:
len(all_dates)

In [ ]:
# format lists to store as dataframe
dict = {'Date': all_dates, 'Title': all_titles}
df = pd.DataFrame(dict)
#Source: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
df['Date'] = pd.to_datetime(df['Date'])
df.head()

In [ ]:
#Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html
# sort articles by pub date
df = df.sort_values(by='Date', ascending=True)
df.head()

In [ ]:
# check for missing values
df.isna().sum()

In [ ]:
# save dataframe to csv file
df.to_csv("GME_news_data.csv", index=False)